### Прочитали fasta формат

In [1]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("sequence.fasta", format="fasta")


In [37]:
record.seq[0:127]

Seq('MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYET...DIT', SingleLetterAlphabet())

### Рассмотрим результат BLAST поиска

In [173]:
result_handle = open('W8ZR3WEN015-Alignment.xml')

In [174]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)

In [175]:
good_almns = {}
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            titles = alignment.title.split('>')
            for title in titles:
                t = title.split('|')
                ss =['517848989']
                for s in ss:
                    if t[1] == s:
                        print("****Alignment****")
                        print("sequence:", alignment.title)
               


Варианты, которые получлсь у меня и совпали со статьей 504822023, 505222758, 

In [1]:
good_almns

In [2]:
"""for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print(hsp.query_start, hsp.query_end, sep = ' ')
            print(hsp.query[0:75] + '...')"""

'for blast_record in blast_records:\n    for alignment in blast_record.alignments:\n        for hsp in alignment.hsps:\n            print("****Alignment****")\n            print("sequence:", alignment.title)\n            print(hsp.query_start, hsp.query_end, sep = \' \')\n            print(hsp.query[0:75] + \'...\')'

### Скачивание OneKP

In [4]:
from bs4 import BeautifulSoup   # Для обработки HTML
import requests # для получения кода страницы

Пропарсила страницу нашла нужные ссылки

In [5]:
data = requests.get('http://www.onekp.com/public_data.html')

In [9]:
soup = BeautifulSoup(data.text, 'html.parser')

In [1]:
print(soup.prettify())

Получим все ссылки

In [28]:
s= []
for string in soup.find_all('a'):
    s.append(string.get('href'))

Будем обращаться для скачивания по необходимым ссылкам

In [57]:
import urllib

In [55]:
res = [elem if elem.find("Trans-assembly")!=-1 else None for elem in s]

Попытка (удачная) скачать файл

In [59]:
url = 'http://206.12.96.204/onekp/URDJ-SOAPdenovo-Trans-assembly.fa.bz2'
urllib.request.urlretrieve(url, "URDJ.bz2")

('URDJ.bz2', <http.client.HTTPMessage at 0x7f476baf3e10>)

### Разделение файла с белками из Interpro

In [3]:
with open('proteins-matched_interpro.txt', 'r') as file:
    n = 0
    s_1, s_2, s_3, s_4 = [], [], [], []
    for line in file:
        if line.startswith('>'):
            n += 1
        if n <= 98328:
            s_1.append(line)
        elif 98328 < n <= 98328*2:
            s_2.append(line)
        elif 98328 * 2 < n <= 98328*3:
            s_3.append(line)
        else:
            s_4.append(line)
            
            

In [4]:
with open('protein_1.txt','w') as file:
    file.write(''.join(s_1))
with open('protein_2.txt','w') as file:
    file.write(''.join(s_2))
with open('protein_3.txt','w') as file:
    file.write(''.join(s_3))
with open('protein_4.txt','w') as file:
    file.write(''.join(s_4))

### Парсинг файлов mast и meme 

In [121]:
from bs4 import BeautifulSoup
with open("Without_Cys/mast_interpro_4_cl.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')
#print(soup.prettify())

In [100]:
motifs = soup.motifs.findAll('motif')

In [101]:
length = [motif['length'] for motif in motifs]

In [102]:
length

['35', '31', '29']

На 40 месте  в мотиве стоит **С** на который и нужно проверить или для Cysless на 24 месте не должен стоять.

In [122]:
sequences = soup.sequences.findAll('sequence')

In [104]:
import re
reg = re.compile('[^a-zA-Z ]')

Нашли все имена таких сиквенсов и также запсали данные о них в формати xml

In [123]:
Cys_seq_names = set()
Cys_seq = set()
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True:
            if reg.sub('', seg.data.string)[int(seg.find(idx = '0')['pos']) - int(seg['start']) + 23] != 'C':
                Cys_seq_names.add(seq['name'])
                Cys_seq.add(seq)

In [59]:
with open('Without_Cys/Cysless_sequences_4.xml', 'w') as f:
    for i in Cys_seq:
        f.write(str(i) + '\n')

Все отсеившееся по нашему выбору сиквенсы запишем в отдельный файл

In [124]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [125]:
new_file = []
for name in Cys_seq_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [126]:
with open('Cysless_proteins_4.fasta', 'w') as f:
    f.write(''.join(new_file))

### Парсинг результатов pfam 

Сначала вручную удаляем информацию с *#* кроме последнего, с наименованием величин.  

In [83]:
import pandas as pd

In [90]:
columns = []
d = []
with open('pfam.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

In [97]:
data = pd.DataFrame(d, columns=columns)

In [98]:
data

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,A0A0H2XCS3,13,75,13,120,PF08446.11,PAS_2,Domain,1,63,107,48.1,1.6e-12,1,CL0183
1,A0A0H2XCS3,148,305,145,308,PF01590.26,GAF,Domain,5,130,133,67.0,2.5e-18,1,CL0161
2,A0A0H2XCS3,325,510,323,512,PF00360.20,PHY,Family,3,181,183,180.3,2.2e-53,1,CL0161
3,A0A0H2XCS3,539,632,528,632,PF13426.7,PAS_9,Domain,8,101,104,26.9,4.4e-06,1,CL0183
